# 0. Read XML and get measures to invert chords

### A. Parse tree

In [1]:
import xml.etree.cElementTree as ET

filename = "XMLs/至少還有你_主旋律.musicxml"
tree = ET.parse(filename)

In [2]:
root = tree.getroot()
root.tag, root.attrib

('score-partwise', {'version': '3.1'})

In [3]:
all_notes = root.findall('.//note')

### B. Read measures

In [9]:
from mxlParsing import *

xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    measures.append(measure)

In [10]:
for measure in measures:
    measure.printMeasure()

Note: 0 / None / None
Note: 0 / None / None
Note: 0 / None / None
Note: 0 / whole / None
Note: 0 / whole / None
Note: 0 / whole / None
Note: 0 / None / None
Note: 0 / half / None
Note: 0 / quarter / None
Note: 0 / eighth / None
Note: G4 / sixteenth / None
Note: G4 / sixteenth / None
Note: F4 / eighth / None
Note: A4 / sixteenth / None
Note: D5 / sixteenth / None
Note: D5 / half / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: B4 / sixteenth / None
Note: B4 / half / None
Note: B4 / eighth / None
Note: G4 / sixteenth / None
Note: G4 / sixteenth / None
Note: 0 / None / None
Note: F4 / eighth / None
Note: A4 / eighth / None
Note: D5 / eighth / None
Note: E5 / sixteenth / None
Note: E5 / sixteenth / None
Note: E5 / sixteenth / None
Note: D5 / eighth / None
Note: 0 / eighth / None
Note: D5 / sixteenth / None
Note: E5 / sixteenth / None
Note: F5 / eighth / None
Note: C5 / sixteenth / None
Note: C